In [1]:
# import packages
!pip install faker
!pip install names
!pip uninstall -y dist/OpenSpecimenAPIconnector-0.9.1-py3-none-any.whl
!pip install dist/OpenSpecimenAPIconnector-0.9.1-py3-none-any.whl

import json
import pandas as pd
import faker
import os
from IPython.display import display
import requests
import random
import ipywidgets as wid
import numpy as np

Found existing installation: OpenSpecimenAPIconnector 0.9.1
Uninstalling OpenSpecimenAPIconnector-0.9.1:
  Successfully uninstalled OpenSpecimenAPIconnector-0.9.1
Processing ./dist/OpenSpecimenAPIconnector-0.9.1-py3-none-any.whl


In [2]:
import OpenSpecimenAPIconnector as OSconn
import OpenSpecimenAPIconnector.os_core as os_core
import OpenSpecimenAPIconnector.os_util as os_util
import OpenSpecimenAPIconnector.mg_core as mg_core
import OpenSpecimenAPIconnector.mg_util as mg_util

In [3]:
def csv_download_link(df, csv_file_name, delete_prompt=False):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    from IPython.core.display import display, HTML
    link = "http://localhost:8888/edit/work/" + csv_file_name
    df.to_csv(csv_file_name, index=False)
    text = "Download/Edit " + csv_file_name
    display(HTML("<a href={} target=_blank>{}</a>".format(link, text)))
   
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        import os
        os.remove(csv_file_name)

In [4]:
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

def common_filtering(year, purpose):
    output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif (year == ALL):
        common_filter = df_london[df_london.purpose == purpose]
    elif (purpose == ALL):
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & 
                                  (df_london.purpose == purpose)]
    
    with output:
        display(common_filter)

def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value)

def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new)

In [5]:
def multi_checkbox_widget(options_dict):
    """ Widget with a search field and lots of checkboxes """
    search_widget = wid.Text()
    output_widget = wid.Output()
    options = [x for x in options_dict.values()]
    options_layout = wid.Layout(
        overflow='auto',
        border='1px solid black',
        width='300px',
        height='300px',
        flex_flow='column',
        display='flex'
    )
    
    #selected_widget = wid.Box(children=[options[0]])
    options_widget = wid.VBox(options, layout=options_layout)
    #left_widget = wid.VBox(search_widget, selected_widget)
    multi_select = wid.VBox([search_widget, options_widget])

    @output_widget.capture()
    def on_checkbox_change(change):
        
        selected_recipe = change["owner"].description
        #print(options_widget.children)
        #selected_item = wid.Button(description = change["new"])
        #selected_widget.children = [] #selected_widget.children + [selected_item]
        options_widget.children = sorted([x for x in options_widget.children], key = lambda x: x.value, reverse = True)
        
    for checkbox in options:
        checkbox.observe(on_checkbox_change, names="value")

    # Wire the search field to the checkboxes
    @output_widget.capture()
    def on_text_change(change):
        search_input = change['new']
        if search_input == '':
            # Reset search field
            new_options = sorted(options, key = lambda x: x.value, reverse = True)
        else:
            # Filter by search field using difflib.
            #close_matches = difflib.get_close_matches(search_input, list(options_dict.keys()), cutoff=0.0)
            close_matches = [x for x in list(options_dict.keys()) if str.lower(search_input.strip('')) in str.lower(x)]
            new_options = sorted(
                [x for x in options if x.description in close_matches], 
                key = lambda x: x.value, reverse = True
            ) #[options_dict[x] for x in close_matches]
        options_widget.children = new_options

    search_widget.observe(on_text_change, names='value')
    display(output_widget)
    return multi_select

In [6]:
def display_search_select_box(cps_dict):
    descriptions = [key for key in  cps_dict.keys()]

    options_dict = {
        x: wid.Checkbox(
            description=x, 
            value=False,
            style={"description_width":"0px"}
        ) for x in descriptions
    }

    def f(**args):

        results = [key for key, value in args.items() if value]
        display(results)

    ui = multi_checkbox_widget(options_dict)
    out = wid.interactive_output(f, options_dict)
    display(wid.HBox([ui, out]))
    return ui

In [7]:
# The URL of the Openspecimen
print('Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter')
base_url= 'http://biobank-7-2.silicolab.bibbox.org' 
base_url+='/openspecimen/rest/ng'
print('The whole URL for API Calls is: ' + base_url)

# Loginname and Pasword of the API-User, recommended is an extra API User
print('Loginname of the user:')
loginname='admin'#input() 
print('Password of the user:')
password='Login@123'#input()

auth = (loginname, password)
# Setting the login data for the given Openspecimen instance
OSconn.config_manager.set_login(base_url, auth)
print(OSconn.config_manager.set_token)
print(OSconn.config_manager.get_url())
print(OSconn.config_manager.get_auth())
# trying to set the login data again will produce an error except the reset Parameters is set to true
# OSconn.config_manager.set_login(base_url, auth) # will produce assertion error
# OSconn.config_manager.set_login(base_url, auth, reset=True) will work

Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter
The whole URL for API Calls is: http://biobank-7-2.silicolab.bibbox.org/openspecimen/rest/ng
Loginname of the user:
Password of the user:
False
http://biobank-7-2.silicolab.bibbox.org/openspecimen/rest/ng
('admin', 'Login@123')


The next step is to choose the entity which will be created or updated. The following schemas are in Openspecimen available: (Name, openspecimen entity). One have to write the openspecimen entity. ('Collection Protocol', 'cp') ('Collection Protocol Registration', 'cpr') ('User', 'user') ('User Roles', 'userRoles') ('Site', 'site') ('Shipments', 'shipment') ('Institute', 'institute') ('Distribution Protocol Requirement', 'dpRequirement') ('Distribution Protocol', 'distributionProtocol') ('Distribution Order', 'distributionOrder') ('Containers', 'storageContainer') ('Container Type', 'storagecontainerType') ('Container Shipment', 'containerShipment') ('Collection Protocol Event', 'cpe') ('Master Specimen', 'masterSpecimen') ('Participant', 'participant') ('Specimen Requirement', 'sr') ('Visit', 'visit') ('Specimen Aliquot', 'specimenAliquot') ('Specimen Derivate', 'specimenDerivative') ('Specimen Disposal', 'specimenDisposal') ('Consent', 'consent')

In [8]:
schema=wid.Dropdown(
    options= [(), ('Collection Protocol', 'cp'), ('Specimen', 'specimen'), ('Collection Protocol Registration', 'cpr'),
              ('User', 'user'), ('User Roles', 'userRoles'), ('Site', 'site'), ('Shipments', 'shipment'), 
              ('Institute', 'institute'), ('Distribution Protocol Requirement', 'dpRequirement'), 
              ('Distribution Protocol', 'distributionProtocol'), ('Distribution Order', 'distributionOrder'), 
              ('Containers', 'storageContainer'), ('Container Type', 'storagecontainerType'),
              ('Container Shipment', 'containerShipment'), ('Collection Protocol Event', 'cpe'),
              ('Master Specimen', 'masterSpecimen'), ('Participant', 'participant'), ('Specimen Requirement', 'sr'),
              ('Visit', 'visit'), ('Specimen Aliquot', 'specimenAliquot'), 
              ('Specimen Derivate', 'specimenDerivative'),('Specimen Disposal', 'specimenDisposal'),
              ('Consent', 'consent')],
    value=(),
    description='Entity:')

operation = wid.Dropdown(
    options = ["CREATE",  'UPDATE'],
              value="CREATE",
              description='Operation:')

sel_schema = schema.value
def change_value_schema(change):
    global sel_schema
    if change.new:
        sel_schema = change.new[1]
    else: sel_schema = None

op = operation.value
def change_value_op(change):
    global op
    if change.new:
        op = change.new
    else: op = None

schema.observe(change_value_schema, names="value")
operation.observe(change_value_op, names="value")

print('Choose your Entity:')
display(schema)
# operation is only needed for csv import not for aggregating fields
print('Choose Operation:')
display(operation)

Choose your Entity:


Dropdown(description='Entity:', options=((), ('Collection Protocol', 'cp'), ('Specimen', 'specimen'), ('Collec…

Choose Operation:


Dropdown(description='Operation:', options=('CREATE', 'UPDATE'), value='CREATE')

In [9]:
print("Selected Schema: ", sel_schema)
print("Selected Operation: ", op)

#initialize csv_bulk, bulk_operations and CSV_exporter
csv_files= os_core.csv_bulk()
bulk_op= os_util.bulk_operations()
csv_export = os_util.Export_OP()

Selected Schema:  cpr
Selected Operation:  CREATE


In [10]:
# get necessary cp information
cp_tools = os_core.collection_protocol()

# get cp info and wirte a dict in order to simplify the api output
cps = cp_tools.get_all_collection_protocols()
cps_dict = {}
for i, item in enumerate(cps):
    # print("Cp Title:")
    # print(item["title"])
    # print("Cp ID: ",item["id"], "\n")
    cps_dict[item["title"]] = item["id"]

In [11]:
# choose target CP / Check if matching is correct; ui is needed for output

ui = display_search_select_box(cps_dict)

Output()

In [12]:
# get selected cp_ids
cp_ids = []
selected_options = [widget.description for widget in ui.children[1].children if widget.value]
for item in selected_options:
    cp_ids.append(cps_dict[item])

print("CP ids selected for further usage: ", cp_ids)
print()

#cp_tools = os_core.collection_protocol()
#test = cp_tools.get_cp_def(cp_ids[0])
#test2 = cp_tools.get_collection_protocol(cp_ids[0])
#r1 = cp_tools.update_collection_protocol(cp_ids[0], json.dumps(test))
#r2 = cp_tools.update_collection_protocol(cp_ids[0], json.dumps(test2))

aggregator = os_util.aggregator()
aggregator.run_aggregation(sel_schema, cp_ids)

CP ids selected for further usage:  [17, 18]

Age high:  90.0
Age low:  6.0
Age mean:  47.87386843693422
Age high:  90.0
Age low:  6.0
Age mean:  48.277190332326285
Aggregation sucessfull
